# Libraries

In [9]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install gensim

  Using cached gensim-4.1.2-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install tensorflow

     |████████████████████████████████| 489.6 MB 302 bytes/s          
     |████████████████████████████████| 4.1 MB 3.7 MB/s            
     |████████████████████████████████| 2.1 MB 3.4 MB/s            
     |████████████████████████████████| 1.1 MB 4.3 MB/s            
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 4.5 MB 3.7 MB/s            
     |████████████████████████████████| 1.3 MB 2.6 MB/s            
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
     |████████████████████████████████| 13.3 MB 4.0 MB/s            
  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
     |████████████████████████████████| 126 kB 3.0 MB/s            
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Preparing metadata (setup.py) ... done
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     |

In [12]:
import pandas as pd
import numpy  as np
import string 
import re
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from collections import Counter
#from wordcloud import WordCloud

import nltk
from nltk.corpus import stopwords
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer
# #from nltk.tokenize.treebank import TreebankWordDetokenizer
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
#from nltk.corpus import stopwords

from gensim.utils import simple_preprocess
from gensim.models import Word2Vec
import gensim.downloader as api

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

#import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')


from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Sequential, layers
from tensorflow.keras.callbacks import EarlyStopping

print('Done')



ModuleNotFoundError: No module named 'tensorflow'

# Data

In [8]:
pd.set_option('max_colwidth', None) 
df = pd.read_csv("~/Downloads/banData/MeTooHate.csv")[["text", "category"]]
#df.head()

## Undersample data

In [9]:
df1 = df[df["category"]== 1]
df1 = df[df["category"]== 1].dropna()
df0 = df[df["category"]== 0].dropna()
df = pd.concat([df0.sample(df1.shape[0]), df1], axis = 0)


#shuffle rows
df = df.sample(frac=1)
df.shape
df.reset_index(inplace = True)

## Select sample size

In [10]:
N = 100
df = df.head(N)
df["category"].value_counts()

1    56
0    44
Name: category, dtype: int64

In [11]:
y = df["category"]
X = df.drop(columns = ["category"]) 

## Clean data

In [12]:
def clean_data(data):
    
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)

    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
   
    # tokenize + remove scpecial characters + set to lower case
    data = text_to_word_sequence(data) 
    
    # Remove stopwords
    stop_words = set(stopwords.words('english')) 
    data = [w for w in data if not w in stop_words]         
    
    # Remove digits
    data = ' '.join(word for word in data if not word.isdigit())
    
    
    return text_to_word_sequence(data)

def apply_data_cleaning(X, text, drop_text = False):
    ln = X.shape[0]
    sentences = []
    for i in range(ln):
        tmp = X.iloc[i][f'{text}']
        tmp_clean = clean_data(tmp)
        sentences.append(tmp_clean)
    X["sentences"] = sentences
    if drop_text == True:
        X.drop(columns = f'{text}', inplace = True)
    return X

In [13]:
X = apply_data_cleaning(X = X, text = "text", drop_text = True) 

## Train-Test split


In [14]:
X = list(X.sentences)
y = np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

80
80
20
20


## Tokenize  data

In [ ]:
# Initializes a Keras utilities that does all the tokenization for you
tokenizer = Tokenizer()

# The tokenization learns a dictionnary that maps a token (integer) to each word
# It can be done only on the train set - we are not supposed to know the test set !
# This tokenization also lower your words, apply some filters, and so on - you can check the doc if you want
tokenizer.fit_on_texts(X_train)
    
# We apply the tokenization to the train and test set
X_train_token = tokenizer.texts_to_sequences(X_train)
X_test_token = tokenizer.texts_to_sequences(X_test)
#X_test_token

In [ ]:
## sjekk
# sentence_number = 10

# input_raw = X_train[sentence_number]
# input_token = X_train_token[sentence_number]
# for i in range(2):
#     print(f'Word : {input_raw[i]} -> Token {input_token[i]}')

### Add vocabulary

The dictionary that maps each word to a token can be accessed with `tokenizer.word_index`
    
Add a `vocab_size` variable that stores the number of different words (=tokens) in the train set. This is called the _size of the vocabulary_.

In [ ]:
vocab_size = len(tokenizer.word_index)

print(f'There are {vocab_size} different words in the train set')

## Padding
_filling_ cells

In [ ]:
X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post')

!set maxlen <<< lower than longest sentence for efficiency/wt loss info

In [ ]:
def plot_hist(X):
    len_ = [len(_) for _ in X]
    plt.hist(len_)
    plt.title('Histogram of the number of sentences that have a given number of words')
    plt.show()
    
plot_hist(X_train);

In [ ]:
X_train_pad = pad_sequences(X_train_token, dtype='float32', padding='post', maxlen=20)
X_test_pad = pad_sequences(X_test_token, dtype='float32', padding='post', maxlen=20)

# RNN - embedding trained on corpus

Let's now feed this data to a Recurrent Neural Network.

model:
- an embedding layer whose `input_dim` is the size of your vocabulary (= your `vocab_size`), and whose `output_dim` is the size of the embedding space you want to have
- a RNN (SimpleRNN, LSTM, GRU) layer
- a Dense layer
- an output layer

⚠️ **Warning** ⚠️ Here, you don't need a masking layer. Why? Because `layers.Embedding` has a argument to do that directly, which you have to set with `mask_zero=True`. That also means that your data **HAVE TO** be padded with **0** (which is the default behavior). See the [documentation](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding#example_2) to understand how it **impacts** the `input_dim`.

!`input_dim` should equal size of vocabulary + 1


Compile it with the appropriate arguments

In [ ]:
embedding_dimension = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size + 1, output_dim=embedding_dimension, mask_zero=True))
model.add(layers.LSTM(20))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
#print(f'Expected number of parameters : {(vocab_size + 1) * embedding_dimension}')
es = EarlyStopping(patience=4, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          epochs=20, 
          batch_size=16,
          validation_split=0.3,
          callbacks=[es]
         )

In [ ]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

# NN Emebdding with Word2Vec - understanding

In [1]:
X_train.head[1000]

NameError: name 'X_train' is not defined

In [ ]:
word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv
#wv["familiar"] --> in X_train but not wv??
size = len(wv['harassment'])
size

In [ ]:
wv.most_similar('harassment')
#word_embedding = wv[''harassment']
#wv.similar_by_vector(word_embedding)

In [ ]:
print('Vocabulary size', len(wv.key_to_index))
diff_words = set([_ for elt in X_train for _ in elt])
print('Number of different words in the train set', len(diff_words))

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it chooses many splits in the different sentences, choose some words as inputs  𝑋  and a word as output  𝑦  which it tries to predict, in the embedding space.

And as any neural network, Word2Vec has some hyperparameters.

## Word2Vec hyperparameters


The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

## vector_size

In [ ]:
word2vec_size30 = Word2Vec(sentences=X_train, vector_size = 30)
wv_size30 = word2vec_size30.wv
wv_size30.most_similar('harassment')

In [ ]:
# print('Vocabulary size', len(wv.key_to_index))
# diff_words = set([_ for elt in X_train for _ in elt])
# print('Number of different words in the train set', len(diff_words))

In [ ]:
word2vec_size50 = Word2Vec(sentences=X_train, vector_size = 50)
wv_size50 = word2vec_size50.wv
wv_size50.most_similar('harassment')

In [ ]:
word2vec_size100 = Word2Vec(sentences=X_train, vector_size = 100)
wv_size100 = word2vec_size100.wv
wv_size100.most_similar('harassment')

## min_count
Second hyperparameter `min_count`,  min # of times a word has to occur in order to be in embedding space

In [ ]:
# min_count:  min # of times a word has to occur in order to nbe in embedding space
word2vec3 = Word2Vec(sentences=X_train, vector_size = 30, min_count = 1)
wv3 = word2vec.wv
wv3.most_similar('harassment')

In [ ]:
## wv3["familiar"] -- > syill not in wv, why?
print('Vocabulary size', len(wv3.key_to_index))
diff_words = set([_ for elt in X_train for _ in elt])
print('Number of different words in the train set', len(diff_words))

In [ ]:
word2vec_0 = Word2Vec(sentences=X_train, vector_size=50, min_count=1)
word2vec_1 = Word2Vec(sentences=X_train, vector_size=50, min_count=2)
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50, min_count=3)
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=5)
word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=10)

In [ ]:
word2vec_1.wv["harassment"]
len(word2vec_1.wv.key_to_index)

In [ ]:
print(f'Number of word in W2V #1 : {len(word2vec_0.wv.key_to_index)}')
print(f'Number of word in W2V #1 : {len(word2vec_1.wv.key_to_index)}')
print(f'Number of word in W2V #2 : {len(word2vec_2.wv.key_to_index)}')
print(f'Number of word in W2V #3 : {len(word2vec_3.wv.key_to_index)}')
print(f'Number of word in W2V #4 : {len(word2vec_4.wv.key_to_index)}')

In [ ]:
diff_words = set([_ for elt in X_train for _ in elt])
len(diff_words)

In [ ]:
word2vec_3.wv.most_similar("harassment")

## window
The arguments you have seen (vector_size, min_count and window) are usually the one that you should start changing to get a better performance for your model.

Other hyperparameters in doc (cf 8.)

In [ ]:
word2vec_3_window1 = Word2Vec(sentences=X_train, vector_size=50, min_count=3, window = 1)
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=3, window = 5) # default 5
word2vec_3_window8 = Word2Vec(sentences=X_train, vector_size=50, min_count=3, window = 8)

In [ ]:
# print(f'Number of word in W2V #window 1 : {len(word2vec_3_window1 .wv.key_to_index)}')
# print(f'Number of word in W2V #window 5: {len(word2vec_3.wv.key_to_index)}')
# print(f'Number of word in W2V #window 8 : {len(word2vec_3_window8 .wv.key_to_index)}')

In [ ]:
#word2vec_3_window1.wv.most_similar("harassment")

In [ ]:
#word2vec_3.wv.most_similar("harassment") # default 5

In [ ]:
#word2vec_3_window8.wv.most_similar("harassment")

# Embedding: convert W2V

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here :

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

## Embedding function

- each word --> vector w2v
- each sentence ---> matrix of w2v 
- e.g sentence of 5 words and embedding size 10 ---> matrix  5x10 --> # of words>=min_count x vector_size

In [ ]:
def embed_sentence(word2vec, sentence):
    """ 
    sentence to matrix
    """
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)
   

def embedding(word2vec, sentences):
    """ 
    list(sentences in words)---> list(matrices)
    """
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [ ]:
sentence = X_train[0]
word2vec = Word2Vec(sentences = X_train, vector_size=50, min_count = 2, window = 6)

print(embed_sentence(word2vec, sentence).shape)
#embed_sentence(word2vec, sentence)  ## --> 4 words in 50D --> 4x50 mx 

In [ ]:
X_train_embedded = embedding(word2vec, X_train)
print(type(X_train))
print(len(X_train), len(X_train_embedded))
print(f'dim of first sentence  {X_train_embedded[0].shape}')
print(f'dim of 2nd  sentence  {X_train_embedded[1].shape}')

## Padding

In order to have ready-to-use data, do not forget to pad them in order to have tensors that can be divided in batch sizes during the optimization. Store the padedd values in X_train_pad and X_test_pad. Do not forget the important arguments of the padding ;)

#! padding with zeros and in the end ?

In [ ]:
# embedd
X_train_embedded = embedding(word2vec, X_train)
X_test_embedded = embedding(word2vec, X_train)

assert(len(X_train_embedded) == len(X_train))
len(X_train_embedded) # list of 80 matrices of xxx rows and 50 columns


In [ ]:
# pad
X_train_pad = pad_sequences(X_train_embedded, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test_embedded, dtype='float32', padding='post')

assert(len(X_train_pad.shape) == 3)
X_train_pad.shape  ## array 80 matrices of 16x50 or 50x16? 
#!16 lmax length of senetnces in X_train

# Modelling NN

## Prep data

In [ ]:
df.head()

In [ ]:
df["category"].value_counts()

In [ ]:
# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)


for X in [X_train_pad, X_test_pad]:
    assert type(X) == np.ndarray
    assert X.shape[-1] == word2vec.wv.vector_size


assert X_train_pad.shape[0] == len(X_train)
assert X_test_pad.shape[0] == len(X_test)

## Baseline model

In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)
print('Baseline accuracy: ', {1/2})
print('Acurracy to bit: ', {(4866 + 64190) / 74712})
#y_pred = 0 if counts[0] > counts[1] else 1
#print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))


## Initial RNN model 
use the NN model, and train with my data?


In [14]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['Recall'])
    
    return model

model = init_model()

NameError: name 'Sequential' is not defined

In [ ]:
es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 32,  # low slower?
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )

## Recall à checker > à Kaggle Recall score: 0.548914858096828

In [ ]:
res = model.evaluate(X_test_pad, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

In [ ]:
#model.summary()

## Improve RNN with transfer learning
use embedding done on other corpus 
Use trained NN with others data?

In [15]:
## list existing models (trained on >>>> data)
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# pick glove-wiki-gigaword-50 for now, will try others--> teachers recommendation?
word2vec_transfer = api.load("glove-wiki-gigaword-50")

In [ ]:
print(len(word2vec_transfer.key_to_index))
print(len(word2vec_transfer['harassment']))
word2vec_transfer['harassment']

In [ ]:
word2vec_transfer.most_similar('harrassment')

In [ ]:
# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

# Pad the training and test embedded sentences
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [ ]:
# Run new model

es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()


In [ ]:
model.fit(X_train_pad_2, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es]
         )
## A dépasser > Recall score: 0.548914858096828

In [ ]:
model.summary()

In [ ]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

# Question to teachers

- Lementazing needed? yes--> reduce vocabulary_size in X_train
- Removing stop_words ---> yes.
- Why words in my X_train do not have embeddings with Word2Vec  --> depends on min_count
- keep (top 20) emojis by replacing with some word  instead of removing in data prep? 
- should we remove numbers in words? ex. "one"

- vector_size effect on embedding, not clear?---> seems like higher dimension better on most_similar words
-! transfer embedding vs "regular" embedding? 
TE = use pre-trained model to represent my data in N-dim
RE = use pre-defined NN to train on my data and represent in som N-dim ?

# Notes for myself

- !! some texts truncated example "yeah...." in sjekk
- remove "amp"--> stands for &
- Add confusion matrix

In [ ]:
# plt.figure(figsize=(6,6))
# cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# sns.heatmap(cm, square=True, annot=True, cbar=False,
#             xticklabels=['non-hate', 'hate'], yticklabels=['non-hate', 'hate'])
# plt.xlabel('Predicted label')
# plt.ylabel('True label')

## Transfer learning vs "simple" embedding

- As mentionned ealier, Word2vec trains an internal Neural network whose goal is to predict a word in a corpus
based on the words around it. This part of the sentence is called the window.
Its size corresponds to the number of word around word W used to predict this word W

- Instead of learning it on your training set (especially if it is very small), you can directly 
load a pretrained embedding


# Keras documentation

- padding : https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences
- embedding: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding#example_2

# Gensim word2vec documentation

- https://radimrehurek.com/gensim/
- https://radimrehurek.com/gensim/models/word2vec.html#usage-examples
- https://radimrehurek.com/gensim/models/word2vec.html#usage-examples
- hyperparameters: https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus
- transfer learning : gensim-data repository.

# To do

- twick Embedding hyperparameters to improve score
- Understand the other layers and hyperparameters to tune
- Improve data cleaning part (lemantizing?)
- Run all on bigger datasett
- ++++

# Results

### Results_ss100_undersampling

| model_name | Embedding_type | validation_accuracy | test_accuracy|
| --- | --- | --- |---|
| | from scratch | 0.4167 |0.55|
| | w2v | 0.75 |0.65|
|  | w2vTF | 0.7083 |0.70|

In [ ]:
0.6667  60.500